In [1]:
import nltk
import csv

Посмотрим на данные: это отзывы о ресторанах и оценка. Будем решать многоклассовую классификацию. Датасет можно сказать по ссылке: https://www.dropbox.com/s/q5sko5ic9rdio14/classification_train.data?dl=0

In [2]:
!wget https://www.dropbox.com/s/q5sko5ic9rdio14/classification_train.data -N
!ls

--2019-06-05 15:48:42--  https://www.dropbox.com/s/q5sko5ic9rdio14/classification_train.data
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/q5sko5ic9rdio14/classification_train.data [following]
--2019-06-05 15:48:42--  https://www.dropbox.com/s/raw/q5sko5ic9rdio14/classification_train.data
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucafaf884baf87829c3e0070fbe2.dl.dropboxusercontent.com/cd/0/inline/AiO7pYUu3D8Y9RfwFV3fuY-UD_3ZWjeQLCBi0CXM6hEjHmyn8PVsPUNyQtjCO5TzIWlkqr8E766SPzFstoy25XYp_YWX4UhyY7GpKySEVFVpqg/file# [following]
--2019-06-05 15:48:43--  https://ucafaf884baf87829c3e0070fbe2.dl.dropboxusercontent.com/cd/0/inline/AiO7pYUu3D8Y9RfwFV3fuY-UD_3ZWjeQLCBi0CXM6hEjHmyn8PVsPUNyQtjCO5TzIWlkqr8E766SPzFstoy25XYp_YWX4

In [3]:
! head -n 2 classification_train.data

Id	Sentiment	Text
0	1	Incredibly disappointing service. I mean really, really bad.\n\nWe placed an order for delivery at 6:30 pm on a Tuesday night, not the busiest night of the week, I'm sure. We were given an estimate of 30-40 minutes. After an hour my husband called to make sure our order wasn't forgotten. The young girl on the phone said that they were very busy and the driver was on his way to our house (less than a mile from the restaurant) at that time and should arrive in 10 minutes. After another 30 minutes we called back and asked to please cancel the order, after 1 1/2 hours we no longer wanted the food. The girl on the phone shouted at my husband that none of this was her fault and was reluctant to cancel our order. She wanted to charge us for food we never received!\n\nThe food is just not good enough for such poor service. If 18 year old college students can't answers phones and take simple orders don't hire them. It's simple.


In [4]:
! wc -l classification_train.data

102583 classification_train.data


Считаем выборку, поделим на трейн и тест так, чтобы в x_train был raw text

In [5]:
!pip install sklearn

/bin/sh: 1: pip: not found


In [30]:
train_file = csv.reader(open('classification_train.data'), delimiter='\t')
train_set = list(train_file)
_, train_label, train_data = zip(*train_set[1:])
print("label:", train_label[0], "\ndata:", train_data[0], )
from sklearn.model_selection import train_test_split
x_train, x_validate, y_train, y_validate = train_test_split(train_data, train_label, test_size=0.2, random_state=0)

label: 1 
data: Incredibly disappointing service. I mean really, really bad.\n\nWe placed an order for delivery at 6:30 pm on a Tuesday night, not the busiest night of the week, I'm sure. We were given an estimate of 30-40 minutes. After an hour my husband called to make sure our order wasn't forgotten. The young girl on the phone said that they were very busy and the driver was on his way to our house (less than a mile from the restaurant) at that time and should arrive in 10 minutes. After another 30 minutes we called back and asked to please cancel the order, after 1 1/2 hours we no longer wanted the food. The girl on the phone shouted at my husband that none of this was her fault and was reluctant to cancel our order. She wanted to charge us for food we never received!\n\nThe food is just not good enough for such poor service. If 18 year old college students can't answers phones and take simple orders don't hire them. It's simple.


In [31]:
x_train[0]

'I like this location because they have a drive-thru. Even though there is almost always a long line, they get you on your way fast. The staff is friendly and competent. Also, they rarely run out of anything (other locations seem to go through their entire inventory of breakfast sandwiches and scones by 9am).\\n\\nIf you are the type that does not drink your morning coffee inside a moving vehicle, they also have comfy chairs inside and decent patio seating.  The patio faces the parking lot and drive-thru but it does have shade umbrellas so it can be very pleasant in the morning.'

In [32]:
y_train[0]

'3'

"Тупое" решение:

Посмотрим что будет, если применить самое простое решение: найти 100 самых частотных слов и использовать их в качестве признаков.

- Найдем топ 100 самых частотных слов

In [33]:
from collections import Counter


In [34]:
from collections import Counter

def create_bag_with_freq(data):
    result = Counter()
    for s in data:
        result.update(s.strip().split())
    return list(result.items())
  
train_bag = create_bag_with_freq(x_train)
print('Number of unique "words": ', len(train_bag))



Number of unique "words":  484082


In [35]:
most_frequent_word = sorted(train_bag, key=lambda x: x[1], reverse=True)[:100]
most_frequent_word[:10]

[('the', 654951),
 ('and', 492240),
 ('a', 411657),
 ('I', 385802),
 ('to', 359227),
 ('of', 248340),
 ('was', 240102),
 ('is', 184703),
 ('for', 167194),
 ('in', 162966)]

В наивный байесовский классификатор надо подать текст в определенном формате

In [36]:
def make_bow_sample(bow, sample):
    for s in sample:
        s = s.strip().split()
        yield { word:word in s for word, _ in bow}

In [37]:
bow_train = [(x, y) for x, y in zip(make_bow_sample(most_frequent_word, x_train), y_train)]
bow_validate = [x for x in make_bow_sample(most_frequent_word, x_validate)]

In [38]:
bow_train[0]

({'the': True,
  'and': True,
  'a': True,
  'I': True,
  'to': True,
  'of': True,
  'was': False,
  'is': True,
  'for': False,
  'in': True,
  'it': True,
  'that': True,
  'with': False,
  'but': True,
  'my': False,
  'on': True,
  'you': True,
  'The': True,
  'have': True,
  'this': True,
  'had': False,
  'they': True,
  'not': True,
  'are': True,
  'at': False,
  'were': False,
  'we': False,
  'so': True,
  'be': True,
  'as': False,
  'like': True,
  'place': False,
  'just': False,
  'out': True,
  'or': False,
  'me': False,
  'get': True,
  'food': False,
  'very': True,
  'good': False,
  'all': False,
  'from': False,
  'there': True,
  'their': True,
  'one': False,
  'if': False,
  'about': False,
  'some': False,
  'would': False,
  'up': False,
  'an': False,
  'really': False,
  'our': False,
  'when': False,
  'been': False,
  'which': False,
  'go': True,
  'your': True,
  'can': True,
  'great': False,
  'here': False,
  '-': False,
  'It': False,
  'We': False

Воспользуемся наивным байесовским классификатором. 
Плюс данного классификатора - можно посмотреть какиме слова оказались наиболее полезными.

In [39]:
nb = nltk.NaiveBayesClassifier.train(bow_train)
print(nb.show_most_informative_features())
predicted = [nb.classify(o) for o in bow_validate]

Most Informative Features
                    love = True                5 : 1      =      3.5 : 1.0
                   great = True                5 : 1      =      3.1 : 1.0
                     was = False               5 : 1      =      2.7 : 1.0
                  always = True                5 : 1      =      2.6 : 1.0
                  pretty = True                3 : 1      =      2.5 : 1.0
                      no = True                1 : 5      =      2.4 : 1.0
                      he = True                1 : 4      =      2.4 : 1.0
                      to = False               4 : 1      =      2.3 : 1.0
                  didn't = True                2 : 5      =      2.3 : 1.0
                    nice = True                4 : 1      =      2.2 : 1.0
None


In [40]:
import numpy as np

Замерьте качество с помощью accuracy

In [41]:
print('accuracy', np.mean(np.array(list(map(float,predicted))) == np.array(list(map(float, y_validate)))))

accuracy 0.3787605441513482


In [42]:
import csv
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
import numpy as np
import glob

In [43]:
y_train = list(map(float, y_train))
y_validate = list(map(float, y_validate))

Обучите TfidfVectorizer на трейне, примените на тесте.

Почему надо действовать именно так, почему нельзя обучить и на трейне, и на тесте?

In [ ]:
tfidf = TfidfVectorizer(encoding=u'utf-8', ngram_range=(1, 2), analyzer='word')
Xtrain = tfidf.fit_transform(x_train)
Xtest = tfidf.transform(x_validate)

- Попробуйте LodisticRegression, LinearSVC, SGDClassifier с какой-нибудь функцией потерь.
- При обучении  SGDClassifier не забудьте поставить побольше итераций, так как это итеративный метод
- параметр class_weight='balanced' может быть полезен. Что он означает?
- Можете повариьировать так же параметры TF-IDF vectorizer
- Попробовать прологарифмировать частоты np.log1p(), или другое нелинейное преобразование.

SVM vs LinearSVC
(LinearSVC быстрее, но не выадет вероятностей, а лишь расстояние до решающей границы. Перевести в вероятности можно, откалибровав http://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html)


In [ ]:
lr = LogisticRegression(C=1, n_jobs=-1)
lr.fit(Xtrain, y_train)
lr_pr = lr.predict(Xtest)

Замерьте качество

In [46]:
print('accuracy', np.mean(np.array(list(map(float,lr_pr)))== np.array(list(map(float, y_validate)))))

accuracy 0.5904237164171827


In [47]:
from sklearn.svm import LinearSVC

In [48]:
# clf = [LogisticRegression(n_jobs=-1)
# LinearSVC(C=1, loss='hinge', class_weight='balanced'),
# SGDClassifier(loss='modified_huber', class_weight='balanced', alpha=1e-2, n_iter=50, n_jobs=-1),
# SGDClassifier(loss='squared_hinge', class_weight='balanced', alpha=1e-2, n_iter=50, n_jobs=-1),
# SGDClassifier(loss='hinge',class_weight='balanced', alpha=1e-2, random_state=3, n_iter=50, n_jobs=-1)]

Какой алгоритм сработал лучше свего?

Когда обучаем многоклассовую классификацию для такой задачи, не учитываем то, что метки 1 и 2 более похожи между собой, чем 4 и 5. Как это можно было бы учесть при обучении модели?

Переходим к нейросетевым подходам

In [49]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.preprocessing.text import Tokenizer

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [50]:
x_train[0]

'I like this location because they have a drive-thru. Even though there is almost always a long line, they get you on your way fast. The staff is friendly and competent. Also, they rarely run out of anything (other locations seem to go through their entire inventory of breakfast sandwiches and scones by 9am).\\n\\nIf you are the type that does not drink your morning coffee inside a moving vehicle, they also have comfy chairs inside and decent patio seating.  The patio faces the parking lot and drive-thru but it does have shade umbrellas so it can be very pleasant in the morning.'

In [51]:
TEXT_LENGTH = 100
VOCABULARY_SIZE = 50000
EMBEDDING_DIM = 30
DIMS = 56
MAX_FEATURES = 5000
batch_size = 32

nb_filter = 50
filter_length = 3
hidden_dims = 50
nb_epoch = 5

In [52]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(x_train)
tokenizer

In [53]:
sequences = tokenizer.texts_to_sequences(x_train)
X_train = tokenizer.sequences_to_matrix(sequences, mode='count')
sequences = tokenizer.texts_to_sequences(x_validate)
X_test = tokenizer.sequences_to_matrix(sequences, mode='count')

In [54]:
X_train.shape, X_test.shape

((82035, 5000), (20509, 5000))

In [55]:
y_train = np_utils.to_categorical(y_train)[:, 1:]
y_test = np_utils.to_categorical(y_validate)[:, 1:]

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(MAX_FEATURES,), activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size,  validation_split=0.1)

In [ ]:
pr = model.predict(X_test)
pr_nn = list(np.argmax(pr, axis=1))
print(list(pr_nn))

Замерьте качество

In [58]:
pr = model.predict(X_test)
np.mean(np.argmax(pr, axis=1) == np.argmax(y_test, axis=1))

0.5309376371349164

Теперь пробуем LSTM

In [59]:
from keras.preprocessing import sequence
sequences = tokenizer.texts_to_sequences(x_train)
X_train = sequence.pad_sequences(sequences, maxlen=TEXT_LENGTH)
sequences = tokenizer.texts_to_sequences(x_validate)
X_test = sequence.pad_sequences(sequences, maxlen=TEXT_LENGTH)

In [60]:
max_review_length = max([len(el) for el in X_train])
top_words = 5000

In [61]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 505       
Total params: 213,705
Trainable params: 213,705
Non-trainable params: 0
_________________________________________________________________
None
Train on 82035 samples, validate on 20509 samples
Epoch 1/3
82035/82035 [==============================] - 155s 2ms/step - loss: 0.4120 - acc: 0.8101 - val_loss: 0.3594 - val_acc: 0.8255
Epoch 2/3
82035/82035 [==============================] - 143s 2ms/step - loss: 0.3500 - acc: 0.8318 - val_loss: 0.3553 - val_acc: 0.8270
Epoch 3/3
82035/82035 [==============================] - 149s 2ms/step 

Замерьте качество

In [63]:
pr = model.predict(X_test)
np.mean(np.argmax(pr, axis=1) == np.argmax(y_test, axis=1))

0.5529767419181822

In [64]:
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

Теперь пробуем добавить сверточный слой

In [65]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 505       
Total params: 216,809
Trainable params: 216,809
Non-trainable params: 0
_________________________________________________________________
None
Train on 82035 samples, validate on 20509 samples
Epoch 1/3
82035/82035 [==============================] - 88s 1ms/step - loss: 0.4

Замерьте качество

In [66]:
pr = model.predict(X_test)
np.mean(np.argmax(pr, axis=1) == np.argmax(y_test, axis=1))

0.5611195085084597